In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import openpyxl
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_excel('./data/kroger.xlsx')

df.set_index('date',inplace=True)
df['revenue']=df['revenue'].str.replace(',','').str.replace('$','').astype(float)
df.head()

,revenue
date,
2025-07-31,33940.0
2025-04-30,45118.0
2025-01-31,34308.0
2024-10-31,33634.0
2024-07-31,33912.0


In [3]:
scaler=MinMaxScaler()
df['revenue']=scaler.fit_transform(df[['revenue']].values)
df.head()

,revenue
date,
2025-07-31,0.553713
2025-04-30,0.994052
2025-01-31,0.568210
2024-10-31,0.541658
2024-07-31,0.552610


In [4]:
def create_sequences(data, seq_length):
    X,y=[],[]
    for i in range(len(data)-seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)
seq_length=12
X,y=create_sequences(df['revenue'].values, seq_length)

In [5]:
class TimeSeriesDataset(Dataset):
    def __init__(self,X,y):
        self.X=torch.tensor(X,dtype=torch.float32)
        self.y=torch.tensor(y,dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self,idx):
        return self.X[idx], self.y[idx]

In [6]:
dataset=TimeSeriesDataset(X,y)
loader=DataLoader(dataset,batch_size=16,shuffle=False)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers):
        super(LSTMModel,self).__init__()
        self.lstm=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc=nn.Linear(hidden_size,1)
    def forward(self,x):
        out,(hn,cn)=self.lstm(x)
        out=self.fc(out[:,-1,:])
        return out